In [ ]:
"""
代码块1
导入环境变量和基础配置
"""

# 在原版 OpenAI.ipynb 的基础上修改, 使用 LangChain 调用 API

import os

# pip install python-dotenv
from dotenv import load_dotenv, find_dotenv

# find_dotenv() 寻找并定位 .env 文件的路径
# load_dotenv() 读取该 .env 文件，并将其中的环境变量加载到当前的运行环境中
# 确保每次读取后的参数能够覆盖旧的值, 保证修改能够生效

_ = load_dotenv(find_dotenv(), override=True)

# 配置系统代理
# os.environ['HTTPS_PROXY'] = 'http://127.0.0.1:7890'
# os.environ["HTTP_PROXY"] = 'http://127.0.0.1:7890'

In [ ]:
"""
代码块2
使用 langchain 调用 LLM 的 API
"""

# 对原版 OpenAI API 来说, Kimi 是直接兼容的
# 对于 LangChain 来说, 则不能使用 OpenAI 的 API, 社区提供了解决方案

"""
使用 langchain_openai
import openai
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(temperature=0.0)
"""

from langchain_community.llms.moonshot import Moonshot

api_key = os.environ.get("KIMI_API_KEY")
os.environ["moonshot_api_key"] = api_key

llm = Moonshot(temperature=0.0)
# model_name max_tokens temperature stream 超参数可选
# https://platform.moonshot.cn/docs/guide/utilize-the-streaming-output-feature-of-kimi-api
# stream=True

output = llm.invoke("你好，简单介绍下Moonshot")

output

In [ ]:
"""
代码块3
构建 Prompt
"""

from langchain_core.prompts import ChatPromptTemplate

template = "你是一个翻译助手，可以帮助我将 {input_language} 翻译成 {output_language}."
human_template = "{text}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", human_template),
])

text = "我带着比身体重的行李，\
游入尼罗河底，\
经过几道闪电 看到一堆光圈，\
不确定是不是这里。\
"
messages  = chat_prompt.format_messages(input_language="中文", output_language="英文", text=text)

messages

In [ ]:
"""
代码块4
输出解析器的使用
"""

from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()
output_parser.invoke(output)


In [ ]:
"""
代码块5
使用 LCEL 构建完整流程链
"""

# 类似于Linux的管道, 用法和效果都很像
chain = chat_prompt | llm | output_parser
chain.invoke({"input_language":"中文", "output_language":"英文","text": text})